In [148]:
import pybaseball

In [149]:
from pybaseball import statcast
from pybaseball import playerid_lookup
from pybaseball import statcast_pitcher
from pybaseball import statcast_batter

In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [151]:
from sklearn import preprocessing

In [152]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [153]:
pybaseball.cache.enable()

In [154]:
data = pybaseball.statcast(start_dt='2018-03-29', end_dt='2019-10-05')

This is a large query, it may take a moment to complete
Skipping offseason dates


100%|█████████████████████████████████████████| 414/414 [01:01<00:00,  6.75it/s]


In [8]:
#data.to_pickle("2018_2021_data.pkl")

In [6]:
#data = pd.read_pickle("2018_2021_data.pkl")

In [155]:
data2 = data[(data.game_type =='R') & (data.type=='X')] #only want in play

In [156]:
data2['year'] = pd.DatetimeIndex(data2['game_date']).year

/var/folders/xd/l65my3cs3xd22_hbb7427_1r0000gn/T/ipykernel_91708/3956029374.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['year'] = pd.DatetimeIndex(data2['game_date']).year


In [157]:
data2 = data2[data2.year==2018] #just do 2018 for now

In [158]:
data2['y'] = np.where(data2['woba_value'] > 0, 1, 0) #simple yes or no hit

In [160]:
columns = ['pitch_type','release_speed',  'zone', 
          'vx0', 'vy0', 'vz0', 'ax','ay','az','launch_speed', 'launch_angle', 'hc_x', 'hc_y','y']

In [161]:
data3 = data2[columns]

In [164]:
data3.groupby('y').mean()

,release_speed,zone,vx0,vy0,vz0,ax,ay,az,launch_speed,launch_angle,hc_x,hc_y
y,,,,,,,,,,,,
0,88.781889,6.89076,2.296671,-129.073967,-4.000852,-2.814226,26.221873,-23.38279,85.578915,12.879743,127.959724,133.014065
1,88.876462,6.654282,2.27981,-129.209966,-4.049338,-2.826505,26.192564,-23.344661,93.40566,11.088778,124.388161,101.568882


In [165]:
data34 = data3.iloc[0:5500] # was having issues with size of data, so going to use small dataseet for now

In [167]:
data_vars=data34.columns.values.tolist()
to_keep=[i for i in data_vars]

In [168]:
data_final=data34[to_keep]
data_final.columns.values

array(['pitch_type', 'release_speed', 'zone', 'vx0', 'vy0', 'vz0', 'ax',
       'ay', 'az', 'launch_speed', 'launch_angle', 'hc_x', 'hc_y', 'y'],
      dtype=object)

In [169]:
data_final = data3[['release_speed',   
          'vx0', 'vy0', 'vz0', 'ax','ay','az','launch_speed', 'launch_angle', 'hc_x', 'hc_y','y']].dropna()

In [170]:
X = data_final.loc[:, data_final.columns != 'y']
y = data_final.loc[:, data_final.columns == 'y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [171]:
data_final_vars=data_final.columns.values.tolist()
y=['y']
X=[i for i in data_final_vars if i not in y]
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(X_train, y_train.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

/Users/jackguebert/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_features_to_select=20 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


[ True  True  True  True  True  True  True  True  True  True  True]
[1 1 1 1 1 1 1 1 1 1 1]


In [172]:
import statsmodels.api as sm

In [173]:
X = data_final.loc[:, data_final.columns != 'y']
y = data_final.loc[:, data_final.columns == 'y']

In [174]:
logit_model=sm.Logit(y,X.astype(float))

In [175]:
result=logit_model.fit()
print(result.summary2()) # get rid of anything with P>0.05

Optimization terminated successfully.
         Current function value: 0.542777
         Iterations 6
                          Results: Logit
Model:              Logit            Pseudo R-squared: 0.149      
Dependent Variable: y                AIC:              129885.8168
Date:               2022-02-03 20:52 BIC:              129992.4305
No. Observations:   119629           Log-Likelihood:   -64932.    
Df Model:           10               LL-Null:          -76339.    
Df Residuals:       119618           LLR p-value:      0.0000     
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     6.0000                                        
------------------------------------------------------------------
                  Coef.  Std.Err.     z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
release_speed     0.0923   0.0620    1.4900 0.1362 -0.0291  0.2137
vx0              -0.0006   0.0018   -0.3534 0.7238 -0

In [176]:
#cols = ['ax', 'ay', 'az', 'launch_speed', 'launch_angle', 'hc_x', 'hc_y']
cols = ['ay', 'az', 'launch_speed', 'launch_angle', 'hc_x', 'hc_y']

In [177]:
X=X[cols]
logit_model=sm.Logit(y,X.astype(float))
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.543138
         Iterations 6
                          Results: Logit
Model:              Logit            Pseudo R-squared: 0.149      
Dependent Variable: y                AIC:              129962.1737
Date:               2022-02-03 20:53 BIC:              130020.3266
No. Observations:   119629           Log-Likelihood:   -64975.    
Df Model:           5                LL-Null:          -76339.    
Df Residuals:       119623           LLR p-value:      0.0000     
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     6.0000                                        
------------------------------------------------------------------
                  Coef.  Std.Err.     z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
ay                0.0064   0.0018    3.6339 0.0003  0.0029  0.0098
az               -0.0084   0.0009   -9.5034 0.0000 -0

In [178]:
from sklearn import metrics

In [179]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [180]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

/Users/jackguebert/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [181]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.73


In [182]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[21016  2715]
 [ 6954  5204]]


In [183]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.89      0.81     23731
           1       0.66      0.43      0.52     12158

    accuracy                           0.73     35889
   macro avg       0.70      0.66      0.67     35889
weighted avg       0.72      0.73      0.71     35889

